In [10]:
import rasterio
import pandas as pd
import geopandas as gpd
import os
import rioxarray
import shapely
from tqdm import tqdm
import shutil
from itertools import product
from rasterio import windows
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask, raster_geometry_mask
from shapely.geometry import box

In [11]:
project_path = os.path.abspath("..")
dem_path = os.path.join(project_path, "data/cpb_dem/dem_tiles_big")
tmi_path = os.path.join(project_path, "data/TMI_2011_2019/VA_TMI_2011_2019.shp")

In [12]:
# NAD83 / UTM zone 18N: 26918 (This is the projection of the DEM layers)
# The original TMI's projection is also NAD83 / UTM zone 18N
tmi_gdf = gpd.read_file(tmi_path)

In [13]:
def get_tile_geom(tile_tif, crs=None):
    
    rds = rioxarray.open_rasterio(tile_tif)
    
    if crs is not None:

        assert isinstance(crs, str)
        
        rds_proj = rds.rio.reproject(crs)
        minx, miny, maxx, maxy = rds_proj.rio.bounds()
        geometry = shapely.geometry.box(minx, miny, maxx, maxy, ccw=True)
    
    else:
        
        minx, miny, maxx, maxy = rds.rio.bounds()
        geometry = shapely.geometry.box(minx, miny, maxx, maxy, ccw=True)
    
    return geometry


def get_tiles(ds, width=256, height=256):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

def cropping_dem(ref_img_path, ups_img, outfile):
    
    """
    ref_img_path: input 10m resolution band
    ups_img_path: input low resolution band (rasterio.open() output)
    outfile: output low resolution band with geom alinged with ref_img
    """

    ref_img = rasterio.open(ref_img_path)
    # get the geometry of the reference high resolution band
    geom = box(*ref_img.bounds)
    
#     ups_img = rasterio.open(ups_img_path)
    cropped, crop_transf = mask(ups_img, [geom], crop=True, filled=False, all_touched=False)
    
    c, h, w = cropped.shape
    
    meta = ups_img.meta
    meta['width'], meta['height'] = w, h
    meta['transform'] = crop_transf
    meta["count"] = c

    with rasterio.open(outfile, 'w', **meta) as dst:
        dst.write(cropped)

In [14]:
allfiles = [i for i in os.listdir(dem_path) if i.endswith('tif')]

In [15]:
overlap_dir = os.path.join(project_path, "data/cpb_dem/overlap_dem_tiles_big")
checking_overlap = False

while checking_overlap:
    
    checking_overlap = False
    
    for tile in tqdm(allfiles):
        tile_path = os.path.join(dem_path, tile)
        patch_geom = get_tile_geom(tile_path)
        patch_gdf = tmi_gdf[tmi_gdf.within(patch_geom)]

        if not patch_gdf.empty:
            # move all subtiles that are inter-sect with the CUSP data to a separate folder, the imageries in this folder will be used
            # to create training/validation data

            dest_path = os.path.join(overlap_dir, tile)
            shutil.copyfile(tile_path, dest_path)


In [16]:
overlap_tiles = [i for i in os.listdir(overlap_dir) if i.endswith("tif")]

In [17]:
image_clipping = False

cropped_path_temp = os.path.join(project_path, "data/cpb_dem/dem_tiles_small")
cropped_path_overlap = os.path.join(project_path, "data/cpb_dem/overlap_dem_tiles_small_overlaps")


N = 1024

while image_clipping:
    
    image_clipping = False
    
    for overlap_tile in tqdm(overlap_tiles):
        
        output_filename = overlap_tile.split(".")[0] + "_{}-{}.tif"  #'VRGB_2017_tile_{}-{}.tif'
        overlap_tile_path = os.path.join(overlap_dir, overlap_tile)
        
        with rasterio.open(overlap_tile_path) as inds:

            meta = inds.meta.copy()

            for window, transform in get_tiles(inds, N, N):

                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height

                outpath = os.path.join(cropped_path_temp, output_filename.format(int(window.col_off), int(window.row_off)))

                with rasterio.open(outpath, 'w', **meta) as outds:
                    outds.write(inds.read(window=window))


                patch_geom = get_tile_geom(outpath)
                patch_gdf = tmi_gdf[tmi_gdf.overlaps(patch_geom)]

                if not patch_gdf.empty:

                    # move all subtiles that are inter-sect with the CUSP data to a separate folder, the imageries in this folder will be used
                    # to create training/validation data

                    patch_path = os.path.join(cropped_path_overlap, output_filename.format(int(window.col_off), int(window.row_off)))

                    shutil.copyfile(outpath, patch_path)


In [18]:
gen_overlaps = False

cropped_path_temp = os.path.join(project_path, "data/cpb_dem/dem_tiles_small")
cropped_path_overlap = os.path.join(project_path, "data/cpb_dem/overlap_dem_tiles_small_intersects")

small_tiles = [i for i in os.listdir(cropped_path_temp) if i.endswith("tif")]

while gen_overlaps:
    
    gen_overlaps = False
    
    for small_tile in tqdm(small_tiles):
        
        small_tile_path = os.path.join(cropped_path_temp, small_tile)

        patch_geom = get_tile_geom(small_tile_path)
        patch_gdf = tmi_gdf[tmi_gdf.intersects(patch_geom)]

        if not patch_gdf.empty:

            # move all subtiles that are inter-sect with the CUSP data to a separate folder, the imageries in this folder will be used
            # to create training/validation data

            patch_path = os.path.join(cropped_path_overlap, small_tile)
            shutil.copyfile(small_tile_path, patch_path)


In [10]:
# # Below code can be done by using gdal_merge.py if it is running out of the memory

# mosaic_files = False

# while mosaic_files:
    
#     mosaic_files = False

#     src_files_to_mosaic = [rasterio.open(os.path.join(cropped_path_overlap, i)) for i in os.listdir(cropped_path_overlap) if i.endswith("tif")]
    
#     mosaic, out_trans = merge(src_files_to_mosaic)


# out_meta = src.meta.copy()
# out_mosaic_file = os.path.join(project_path, 'data/cpb_dem/mosaic_dem_new.tif')

# # Update the metadata
# out_meta.update({"driver": "GTiff",
#     "height": mosaic.shape[1],
#     "width": mosaic.shape[2],
#     "transform": out_trans})

# with rasterio.open(out_mosaic_file, "w", **out_meta) as dest:
#     dest.write(mosaic)

In [11]:
# Mosaic all small tiles with gdal_merge.py
# Convert the mosaic file into wgs84 with gdalwarp out.tif mosaic_dem_wgs84.tif -t_srs "+proj=longlat +ellps=WGS84"

In [19]:
# tmi label data
tmi_labels_dir = os.path.join(project_path, "data/NAIP_data_processing/tmi_labels")
tmi_labels = [i for i in os.listdir(tmi_labels_dir) if i.endswith("tif")]
out_mosaic_file = os.path.join(project_path, "data/cpb_dem/mosaic_dem_wgs84.tif")

In [20]:
gen_dem_patch = False

while gen_dem_patch:
    
    gen_dem_patch = False
    
    dem_src = rasterio.open(out_mosaic_file)

    for tmi in tqdm(tmi_labels):

        tmi_file = os.path.join(tmi_labels_dir, tmi)
        out_dem_file = os.path.join(project_path, "data/NAIP_data_processing/dem_patch", tmi)
        cropping_bands(tmi_file, dem_src, out_dem_file)


In [14]:
dem_src.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': None,
 'width': 225115,
 'height': 237524,
 'count': 1,
 'crs': CRS.from_wkt('GEOGCS["unknown",DATUM["Unknown_based_on_WGS84_ellipsoid",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'),
 'transform': Affine(9.943021556206257e-06, 0.0, -77.48173212745465,
        0.0, -9.943021556206257e-06, 38.89102857322552)}

In [17]:
tmi_test = rasterio.open(os.path.join(tmi_labels_dir, tmi_labels[0]))

In [18]:
tmi_test.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 256,
 'height': 256,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(5.998318744046421e-06, 0.0, -76.54978047876965,
        0.0, -5.998318744046421e-06, 36.84168643192865)}

In [24]:
dem_src.meta['dtype']

'float32'